# Flow Matching 公式（精简版）

Flow Matching（FM）学习一个**时间依赖的速度场** $v_\theta(x,t)$，把简单分布（如高斯噪声）连续“流动”到数据分布。

## 1) 连续流（常微分方程）
给定速度场：

$$
\frac{dx}{dt}=v_\theta(x,t),\quad t\in[0,1]
$$

从 $x_0\sim p_0$ 出发，积分到 $t=1$ 得到样本 $x_1$。

---

## 2) 线性插值桥（常见选择）
给定数据样本 $x_1\sim p_{data}$，噪声样本 $x_0\sim p_0$，定义线性插值：

$$
x_t=(1-t)x_0+t x_1
$$

其真实速度是常数：

$$
\frac{dx_t}{dt}=x_1-x_0
$$

---

## 3) Flow Matching 训练目标
让模型在任意 $t$ 上拟合真实速度：

$$
\mathcal{L}=\mathbb{E}_{t\sim U(0,1),\,x_0\sim p_0,\,x_1\sim p_{data}}
\left[\|v_\theta(x_t,t)-(x_1-x_0)\|^2\right]
$$

训练好后，用 ODE 求解器从 $x_0$ 积分到 $x_1$ 完成采样。

In [1]:
# Flow Matching 的数值示例（无模型）
import torch

B, C, H, W = 2, 3, 4, 4

# x0: 噪声分布样本，x1: 数据样本（这里只用随机张量代替）
x0 = torch.randn(B, C, H, W)
x1 = torch.randn(B, C, H, W)

# 采样 t ~ U(0,1)
t = torch.rand(B, 1, 1, 1)

# 线性插值桥
x_t = (1 - t) * x0 + t * x1

# 真实速度场（目标）
v_target = x1 - x0

print("t shape", t.shape)
print("x_t shape", x_t.shape)
print("v_target shape", v_target.shape)

t shape torch.Size([2, 1, 1, 1])
x_t shape torch.Size([2, 3, 4, 4])
v_target shape torch.Size([2, 3, 4, 4])


In [2]:
# Flow Matching 的采样示意（用固定速度场近似）
import torch

B, C, H, W = 1, 1, 1, 1
x = torch.randn(B, C, H, W)

# 假设模型给出的速度场 v_theta(x,t) 近似常数（这里只做示意）
v_theta = torch.ones_like(x) * 0.5

# Euler 积分从 t=0 到 t=1
num_steps = 5
dt = 1.0 / num_steps

for _ in range(num_steps):
    x = x + v_theta * dt

print("sample", x)

sample tensor([[[[-0.9221]]]])
